In [0]:
# Replace with your actual storage account name and key 
spark.conf.set( 
    "fs.azure.account.key.noorawitha.dfs.core.windows.net", 
    "f4ZzF5c4wNAH9I64B3fCYgCoWEpq176kycL0MFiYEGZj156DHwZIF0sw0zlVT1qr67YYvIok3PxR+AStfEv6OA==" 
) 

In [0]:
# Load the books dataset from the silver layer
books = spark.read.parquet(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/processed/books/"
)
 
# Load the authors dataset from the silver layer
authors = spark.read.parquet(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/processed/authors/"
)
# Display the first few records to confirm the data was loaded correctly
books.show(5)
authors.show(5)
# Display the columns and their data types to verify the schema
books.printSchema()
authors.printSchema()

+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+--------------------+---------+---------------+-------------+-----------------+-------------------+----------------+--------------------+--------------------+-------+-------------+-------+--------------------+--------------------+
|      isbn|text_reviews_count|country_code|language_code|      asin|is_ebook|average_rating|kindle_asin|         description|   format|                link|           publisher|num_pages|publication_day|       isbn13|publication_month|edition_information|publication_year|                 url|           image_url|book_id|ratings_count|work_id|               title|title_without_series|
+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+--------------------+---------+---------------+-------------+------

In [0]:
from pyspark.sql.functions import col, length, trim, count, when
 
# Read raw (uncleaned) reviews from the silver layer
reviews = spark.read.parquet(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/processed/reviews/"
)
 
# Peek at rows and schema
reviews.show(5, truncate=False)
reviews.printSchema()
 
# Basic profiling: counts and potential issues
total_rows = reviews.count()
null_review_id = reviews.filter(col("review_id").isNull()).count()
null_book_id   = reviews.filter(col("book_id").isNull()).count()
null_user_id   = reviews.filter(col("user_id").isNull()).count()
null_rating    = reviews.filter(col("rating").isNull()).count()
empty_text     = reviews.filter( (col("review_text").isNull()) | (trim(col("review_text")) ==
"") ).count()
 
print(f"Total rows: {total_rows}")
print(f"NULL review_id: {null_review_id}, NULL book_id: {null_book_id}, NULL user_id: {null_user_id}, NULL rating: {null_rating}")
print(f"Empty/NULL review_text: {empty_text}")

+--------------------------------+-------+--------------------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|review_id                       |book_id|user_id                         |rating|review_text                                                                                                                                                                                                                                                                                                                                                           

In [0]:
from pyspark.sql.functions import col, trim, length
 
# Start from the existing Parquet-loaded DataFrame
# (Assumes you already did: reviews = spark.read.parquet(".../processed/reviews/"))
df = reviews
 
# 1) Drop rows missing critical keys
df = df.filter(
    col("review_id").isNotNull() &
    col("book_id").isNotNull() &
    col("user_id").isNotNull()
)
 
# 2) Enforce rating to be integer in [1..5]
df = df.withColumn("rating_int", col("rating").cast("int"))
df = df.filter(
    col("rating_int").isNotNull() &
    (col("rating_int") >= 1) &
    (col("rating_int") <= 5)
)
 
# 3) Normalize text; drop empty or ultra-short reviews (<10 chars after trim)
df = df.withColumn("review_text", trim(col("review_text")))
df = df.filter(
    col("review_text").isNotNull() &
    (length(col("review_text")) >= 10)
)
 
# 4) De-duplicate by review_id (keep arbitrary first; refine if you have timestamps)
df = df.dropDuplicates(["review_id"])
 
# 5) Select final shape
reviews_clean = df.select(
    "review_id",
    "book_id",
    "user_id",
    col("rating_int").alias("rating"),
    "review_text"
)

In [0]:
# Write the cleaned reviews back to the silver layer (overwrite)
reviews_clean.write.mode("overwrite").parquet(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/processed/reviews/"
)
 
# Sanity check: re-read from disk and inspect schema and a few rows
reviews_verified = spark.read.parquet(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/processed/reviews/"
)
reviews_verified.printSchema()
reviews_verified.show(5, truncate=False)
 
print(f"Written cleaned rows: {reviews_verified.count()}")

root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)

+--------------------------------+-------+--------------------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|review_id                       |book_id|user_id                         |rating|review_text                                                                                                                                            

In [0]:
spark.catalog.clearCache()
spark.sql("CLEAR CACHE")

DataFrame[]

In [0]:
base = "abfss://lakehouse@noorawitha.dfs.core.windows.net"

books = spark.read.parquet(f"{base}/processed/books/")
authors = spark.read.parquet(f"{base}/processed/authors/")
reviews_clean = spark.read.parquet(f"{base}/processed/reviews/")  # already cleaned


In [0]:
from pyspark.sql import functions as F, Window as W

n_auth = authors.count()
if n_auth == 0:
    raise Exception("Authors dataset is empty")

books_tmp = books.withColumn("rn_b", F.row_number().over(W.orderBy("book_id")))
authors_tmp = authors.withColumn("rn_a", F.row_number().over(W.orderBy("author_id")))

book_authors = (
    books_tmp
    .withColumn("join_key", ((F.col("rn_b") - 1) % F.lit(n_auth)) + 1)
    .join(authors_tmp.withColumn("join_key", F.col("rn_a")), "join_key")
    .select("book_id", "author_id")
    .dropDuplicates()
)

book_authors.show(5)
book_authors.printSchema()

/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+-------+---------+
|book_id|author_id|
+-------+---------+
|      1|       10|
|     10|     1000|
|    100|    10000|
|   1000|   100000|
|  10000|  1000007|
+-------+---------+
only showing top 5 rows
root
 |-- book_id: string (nullable = true)
 |-- author_id: string (nullable = true)



In [0]:
reviews_clean.printSchema()


root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)



In [0]:
from pyspark.sql import functions as F

curated_final = (
    reviews_clean.alias("r")
    .join(book_authors.alias("ba"), "book_id", "inner")
    .join(books.select("book_id", "title", "text_reviews_count", "language_code",
                       "publication_year", "publication_month", "publication_day").alias("b"), "book_id", "inner")
    .join(authors.select("author_id", "name").alias("a"), "author_id", "inner")
    .select(
        "r.review_id",
        "r.book_id",
        "b.title",
        "a.author_id",
        "a.name",
        "r.user_id",
        "r.rating",
        "r.review_text",
       F.col("b.language_code").alias("language"),
        F.col("b.text_reviews_count").alias("n_votes"),
       F.to_date(F.concat_ws("-", "b.publication_year", "b.publication_month", "b.publication_day")).alias("date_added")
    )
    .dropDuplicates(["review_id"])
)

curated_final.show(5, truncate=False)
curated_final.printSchema()

/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+--------------------------------+--------+-----------------------------+---------+--------------------+--------------------------------+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
spark.sql("USE CATALOG hive_metastore")
spark.sql("USE SCHEMA default")

curated_final.write.format("delta") \
    .option("overwriteSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("curated_reviews")

print(" Success: curated_reviews schema updated in Hive Metastore!")


/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


 Success: curated_reviews schema updated in Hive Metastore!


In [0]:
spark.sql("DESCRIBE TABLE curated_reviews").show(truncate=False)


+-----------+---------+-------+
|col_name   |data_type|comment|
+-----------+---------+-------+
|review_id  |string   |NULL   |
|book_id    |string   |NULL   |
|title      |string   |NULL   |
|author_id  |string   |NULL   |
|name       |string   |NULL   |
|user_id    |string   |NULL   |
|rating     |int      |NULL   |
|review_text|string   |NULL   |
|language   |string   |NULL   |
|n_votes    |string   |NULL   |
|date_added |date     |NULL   |
+-----------+---------+-------+



In [0]:
%sql
SELECT language, n_votes, date_added
FROM curated_reviews
LIMIT 10;


language,n_votes,date_added
eng,104,2010-12-07
eng,9,2014-06-01
eng,3275,2007-09-18
eng,8,2012-07-17
,42,null
eng,100,null
en-US,446,2012-05-01
en-US,26,2008-11-20
eng,37077,2010-09-13
,9645,2012-11-13


In [0]:
%sql
SELECT *
FROM curated_reviews
LIMIT 10


review_id,book_id,title,author_id,name,user_id,rating,review_text,language,n_votes,date_added
fa241e939d2218940d17cff6fc30bca4,7663760,Fooling Some of the People All of the Time: A Long Short (and Now Complete) Story,5217910,Timo Tiihonen,0c5b36407771dfd65acdd812ecd51705,4,"Investing is an obsession and not a job.I think few exemplify that more than David Einhorn, one of the smartest fundamental investors in the current era. In a world, where short-sellers are castigated as profiteers and trouble makers, Mr Einhorn demonstrates through his most public short that fundamental short-selling is the only tool to expose fraud and corruption in financial markets.His appeal to regulators to close loopholes that allow companies to feed off tax-payer dollars is genuine but it's difficult to be an optimist when incentives are so perverse despite all the outrage in the aftermath of the financial crisis.All I can say is that Mr Einhorn and his ilk, continue to be the de-facto regulators of a broken financial system and if they profit from that, well isn't it just right when you see the breadth of the work they put into doing so.",eng,104,2010-12-07
2628b9004c4b6710aa0c20ce094da2c1,22387890,Deep Water,15138868,Ferne McCann,b78a9143ca2f0c4c7361694dc6cb0500,3,I liked this story despite not being entirely sold on the attraction between Mario and Jordan which result pretty much in insta-love. But it was still a nice short and sweet story.,eng,9,2014-06-01
ec6f00823d47459dc2c70fea9559c605,1096390,The Uncommon Reader,13828874,McKenzie Lewis,c3fa377cfc84401747630b98f92758c9,4,cute story with lots of British humor on the perils of reading too much. Like that could ever happen!,eng,3275,2007-09-18
e995dd91ce98a48306e5be8d15f97c8d,13166894,Death at SeaWorld: Shamu and the Dark Side of Killer Whales in Captivity,197184,John Gruen,ddc44923909c38b4d149a38431105943,4,"This was a good book with a very important message that everybody should at least be aware of. While I agree with a lot of the position this book took,I do have a few notes: - It clearly had an agenda, which is okay, but even though I share the same position on whales in captivity, it is quite biased and only gives a small glimpse into the other side of the debate. - It get's EXTREMELY repetitive. There were points where I had to check to make sure I hadn't lost my place because it felt like I had just read the exact same thing in previous chapters. - Personally, while I was completely interested (especially being that orcas are my favourite animal), there were points, particularly when discussing the legal battle issues where I started to glaze over a little bit and speed read to get through it. All in all though, this was a very thought-provoking read that I overall very much enjoyed.",eng,8,2012-07-17
d6120da6fdef79aaa9e11c623e4b9559,25430624,ABC Dream,408384,T.J. Wray,419dfd723edeb5e27f50aa2382f9aa86,5,"Concept: 5 stars Art: 5 stars Alphabet book with marvelous illustrations that include various items for each letter. In each illustration, readers discern all the words they can that begin with the featured letter, then check their answers against the list in the back of the book. Fun, interactive way to learn the alphabet.",,42,null
601397fde0040a061eda83792cec3653,20501204,"Gathering Storm (Firsts and Forever, #4)",10812,Mario Bellatin,52f1a6e704261d811b3b79424e63e306,4,"It has been a while since I read the previous books of this series, but I found that I easily dropped back into the various characters. I enjoyed the colorful mafia nana and her trope of friends along with the other love bird couples that make an appearance. The main couple was a good match for each other with Hunter being vulnerable and witty and Brian being protective and sarcastic. They made a very adorable couple although their lack of communication problems were a little irritating at times.",eng,100,null
65293457614f7fa067060549f24970d4,12963141,My Stubborn Heart,17120075,Lorain Grabowski,9860a9c252ad0289b9f5308a

In [0]:
spark.sql("DESCRIBE TABLE curated_reviews").show(truncate=False)


+-----------+---------+-------+
|col_name   |data_type|comment|
+-----------+---------+-------+
|review_id  |string   |NULL   |
|book_id    |string   |NULL   |
|title      |string   |NULL   |
|author_id  |string   |NULL   |
|name       |string   |NULL   |
|user_id    |string   |NULL   |
|rating     |int      |NULL   |
|review_text|string   |NULL   |
|language   |string   |NULL   |
|n_votes    |string   |NULL   |
|date_added |date     |NULL   |
+-----------+---------+-------+



In [0]:
gold_path = f"{base}/gold/curated_reviews/"

# Write curated_final as Delta table to GOLD zone
curated_final.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(gold_path)

print("GOLD table saved to:", gold_path)

display(dbutils.fs.ls(gold_path))




/databricks/spark/python/pyspark/sql/connect/expressions.py:1061: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


GOLD table saved to: abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews/


path,name,size,modificationTime
abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews/_delta_log/,_delta_log/,0,1762273074000
abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews/part-00000-293ff527-6817-4be5-aa9f-70ded7235877.c000.snappy.parquet,part-00000-293ff527-6817-4be5-aa9f-70ded7235877.c000.snappy.parquet,43494260,1762622422000
abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews/part-00000-bfba23a9-704a-44ee-bc50-b972bd98e735.c000.snappy.parquet,part-00000-bfba23a9-704a-44ee-bc50-b972bd98e735.c000.snappy.parquet,43487080,1762274727000
abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews/part-00000-c2918015-bdba-4270-87ff-56b595876168.c000.snappy.parquet,part-00000-c2918015-bdba-4270-87ff-56b595876168.c000.snappy.parquet,43175110,1762273150000
abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews/part-00000-da7c6991-4536-4108-aeb3-315951878c4e.c000.snappy.parquet,part-00000-da7c6991-4536-4108-aeb3-315951878c4e.c000.snappy.parquet,43490005,1762274829000
abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews/part-00000-ff3f6d1d-fc8b-45d0-86f7-0f61631f6392.c000.snappy.parquet,part-00000-ff3f6d1d-fc8b-45d0-86f7-0f61631f6392.c000.snappy.parquet,43487858,1762274071000
abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews/part-00001-043cd6c0-bd00-4a8d-9cd6-84d8a5a7095f.c000.snappy.parquet,part-00001-043cd6c0-bd00-4a8d-9cd6-84d8a5a7095f.c000.snappy.parquet,43659555,1762274830000
abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews/part-00001-40d9a29b-ac46-4e19-917c-0761754562b1.c000.snappy.parquet,part-00001-40d9a29b-ac46-4e19-917c-0761754562b1.c000.snappy.parquet,43340838,1762273141000
abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews/part-00001-775230b8-ed8c-4565-b296-e8dd97b256b6.c000.snappy.parquet,part-00001-775230b8-ed8c-4565-b296-e8dd97b256b6.c000.snappy.parquet,43660047,1762274718000
abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews/part-00001-c0055902-11c7-4ce3-9b93-f9726db702f3.c000.snappy.parquet,part-00001-c0055902-11c7-4ce3-9b93-f9726db702f3.c000.snappy.parquet,43654866,1762274063000


In [0]:
%sql
SELECT COUNT(*) AS total_rows
FROM curated_reviews;


total_rows
14971371


In [0]:
display(dbutils.fs.ls(f"{base}/gold/"))


path,name,size,modificationTime
abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews/,curated_reviews/,0,1762273074000
abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews_cleaned/,curated_reviews_cleaned/,0,1762275413000


In [0]:
from pyspark.sql import functions as F

gold_path = "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews/"
df = spark.read.format("delta").load(gold_path)

df.show(5)
df.printSchema()


+--------------------+--------+--------------------+---------+--------------------+--------------------+------+--------------------+--------+-------+----------+
|           review_id| book_id|               title|author_id|                name|             user_id|rating|         review_text|language|n_votes|date_added|
+--------------------+--------+--------------------+---------+--------------------+--------------------+------+--------------------+--------+-------+----------+
|e8b3ad1dc1e566b3e...|25326826|A Willful Romanti...|  3947907|           Beth Camp|7ac68d284be157680...|     5|Disclaimer: I am ...|   en-GB|     16|      NULL|
|1a851c0e6cf754c3d...|   11588|         The Shining| 14742377|Maria Octavian Pa...|198d37b65ef92f865...|     4|I wanted to read ...|     eng|  11351|1980-07-01|
|10fb080f2b16d157e...|16074553|The Apple Orchard...|   475417|       Paul McGuigan|ff4ed175e424528f6...|     3|This is a far-rea...|     eng|   1005|2013-04-30|
|ee63d0b7922892836...|10897866|The

In [0]:
df = df.filter(
    F.col("rating").isNotNull() &
    F.col("review_text").isNotNull()
)


In [0]:
df = df.withColumn("rating", F.col("rating").cast("int"))


In [0]:
df = df.withColumn(
    "word_count",
    F.size(F.split(F.col("review_text"), " "))
)


In [0]:
df = df.withColumnRenamed("name", "author_name")


In [0]:
top_authors = (
    df.groupBy("author_id", "author_name")
      .agg(F.count("*").alias("total_reviews"))
      .orderBy(F.col("total_reviews").desc())
      .limit(10)
)

top_authors.show()

+---------+--------------------+-------------+
|author_id|         author_name|total_reviews|
+---------+--------------------+-------------+
|   150601|   Janet E. Esposito|        20084|
|   587852|       Robert Gryder|        17949|
|  4844000|Elizabeth Svigar ...|        13026|
| 15400711|      Robert A. Lyon|        12912|
|  3203351|        Sugata Mitra|        11421|
|   827276|       Julie Capaldo|        10870|
| 13571859|         Sam Russell|        10423|
|   223867|      Marlene Kliman|        10285|
|   173873|        Bob Carlisle|         9914|
|  6863566|   Vladimir Maliavin|         9292|
+---------+--------------------+-------------+



In [0]:
avg_language = (
    df.groupBy("language")
      .agg(F.avg("rating").alias("avg_rating"))
      .orderBy(F.col("avg_rating").desc())
)

avg_language.show()

+--------+-----------------+
|language|       avg_rating|
+--------+-----------------+
|     yid|              5.0|
|     khm|              5.0|
|     hmn|              5.0|
|     crh|              5.0|
|     chb|              5.0|
|     qaa|              5.0|
|     kok|              5.0|
|     asm|              5.0|
|     sot|              5.0|
|     iku|              5.0|
|     oci|              5.0|
|     tgk|              5.0|
|     non|              5.0|
|     ltz|              5.0|
|     che|              5.0|
|     ach|              5.0|
|     ast|             4.75|
|     bel|4.714285714285714|
|     nav|4.666666666666667|
|     swa|              4.6|
+--------+-----------------+
only showing top 20 rows


In [0]:
top_books = (
    df.groupBy("book_id", "title")
      .agg(F.count("*").alias("n_reviews"))
      .orderBy(F.col("n_reviews").desc())
      .limit(10)
)

top_books.show()

+--------+--------------------+---------+
| book_id|               title|n_reviews|
+--------+--------------------+---------+
|11870085|The Fault in Our ...|    20070|
| 2767052|The Hunger Games ...|    17888|
| 7260188|Mockingjay (The H...|    13025|
|22557272|The Girl on the T...|    12911|
| 6148028|Catching Fire (Th...|    11419|
|   19063|      The Book Thief|    10866|
|10818853|Fifty Shades of G...|    10422|
|13335037|Divergent (Diverg...|    10283|
|   41865|Twilight (Twiligh...|     9906|
|18007564|         The Martian|     9267|
+--------+--------------------+---------+



In [0]:
clean_gold_path = "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews_cleaned/"

df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(clean_gold_path)

print("Clean curated table saved to:", clean_gold_path)


Clean curated table saved to: abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews_cleaned/


In [0]:
# Use Hive Metastore
spark.sql("USE CATALOG hive_metastore")
spark.sql("USE SCHEMA default")

clean_path = "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews_cleaned/"

# Register as SQL table
spark.sql(f"""
CREATE TABLE IF NOT EXISTS curated_reviews_cleaned
USING DELTA
LOCATION '{clean_path}'
""")

print("Table registered successfully")


Table registered successfully


In [0]:
%sql
SHOW TABLES IN hive_metastore.default;


database,tableName,isTemporary
default,curated_reviews,false
default,curated_reviews_cleaned,false
,_sqldf,true


In [0]:
%sql
SELECT *
FROM hive_metastore.default.curated_reviews_cleaned
LIMIT 20;


review_id book_id title author_id author_name user_id rating review_text language n_votes date_added word_count e8b3ad1dc1e566b3e6529e9a6dcee241 25326826 A Willful Romantic (Ganymede Quartet #3) 3947907 Beth Camp 7ac68d284be15768081bd3e86a1431c9 5 Disclaimer: I am a super fangirl of this alt historical M/M romance series and stalk the author, unashamedly, so I can get my next Henry and Martin fix. 
 This is the third novel in the Ganymede Quartet series. It is best to read them in order. 
 Backstory for newcomers: this is an alternate history novel set in NYC in 1901 in a world where slavery exists, is highly regulated and commonplace. Upstanding wealthy families have slave labor to run their homes, and gentlemen/ladies are expected to have companion slaves from age 16, for use with valet duty, homework help, and entertainment--including sex. Gentlemen have male companions, ladies have female companions. Henry is a wealthy boy, and Martin is his companion. 
 Okay. Where we start in this book is New Years' Day with Henry bursting with love for Martin. He so desperately wants to share his joy--it is a wonder to him how intelligent, beautiful Martin can really care for him--over having a companion who is so caring. It troubles Henry that Martin may only love him because it is his job, and often questions Martin about how he might feel if he were a freeman. 
 Martin thinks this is all nonsense. He isn't free, and has no desire to be free. He feels exceptional gratitude to have a master who values him as a person, and shows genuine affection. 
 The two of them are absolutely besotted. Still, this is all behind closed doors. True gentlemen make make use of their slaves, but they are not to kiss them, or ensure their mutual pleasure, or --Heaven forfend--confess to loving them. 
 But Henry is a willful romantic, and a Valentine has been procured. He is still nervous about Martin's close friendship with a fellow slave, Tom, and he desires to know more about Martin's history of training at Ganymede, but the edge of jealousy is wearing off. (Yay!) I really enjoyed Henry's attempts at closeness with Martin. He laments the difference in their station only because slaves are free to be gay, and free men can be (secretly) gay, but a master/slave gay relationship is doubly tricky. Hi attempts at public claiming/closeness grow ever more dangerous. 
 Both Henry and Martin get a bit more voyeuristic in this book, but Henry continues to claim Martin all for himself--never sharing him. There is talk and fantasies about including a third, but Henry's possessive stance soon garners him a bit of recognition, in a good way for a change. Slave relations are definitely in the forefront of this book, and we end with the knowledge that these boys are in for big changes. 
 Cannot wait for the next book! en-GB 16 null 452 1a851c0e6cf754c3de38557c80256606 11588 The Shining 14742377 Maria Octavian Pavnotescu 198d37b65ef92f865df1b71dc317c22c 4 I wanted to read it before I read Doctor Sleep and I am glad I did. Helped me see the differences between the book and the movie. I'm already loving Doctor Sleep and it's great to have The Shining fresh in my mind. 
 The middle of the book dragged a little for me, but the last third was so good I almost didn't care. Strange how older books like this become a time capsule in a way to a period of history you almost forgot. eng 11351 1980-07-01 86 10fb080f2b16d157e39b87487d858f2d 16074553 The Apple Orchard (Bella Vista Chronicles, #1) 475417 Paul McGuigan ff4ed175e424528f64ea534d6fedc5b4 3 This is a far-reaching story that begins with a young woman who works finding lost and ancient treasures and restoring them to their rightful owners. She discovers that she has a grandfather she never met, who has had an accident and is leaving her half of his estate. She travels to find out more about this man and the family she didn't know she had. As the story unfolds, we get a look into the family's history, back into Denmark during 